### Строим запрос к нашему сервису

In [1]:
import pandas as pd
import requests

import urllib.request
import json 

In [2]:
data = ('Load of manure dumped at Democratic headquarters',
        'Print \nA load of manure was dumped outside the Democratic Party headquarters in Warren County. \n“What reasonable person thinks this is OK????” party chair Bethe Goldenfield said in a post in the Greater Cincinnati Politics Facebook Group . “I won’t be responding to anyone who thinks this is acceptable behavior. It is ILLEGAL!” \nThe same thing happened in 2012, Goldenfield noted. The suburban Cincinnati county is overwhelmingly Republican; Mitt Romney got 69 percent of the vote four years ago. It’s been almost 40 years since a Democrat was elected to countywide office. \nGoldenfield told The Enquirer the Warren County Sheriff’s Office called her around 7:45 a.m. Saturday alerting her to the manure pile outside the Lebanon building. Deputies met party officials later to review video. \n“Hopefully the perps will be held accountable for their actions,” she said. \nJeff Monroe, chairman of the Warren County Republican Party, said the GOP had nothing to do with the manure “and has offered to help clean things up.”')

In [3]:
# формируем запрос
def send_json(x):
    title, text = x
    #print(description, company_profile, benefits)
    body = {'title': title, 
            'text': text
            }
    myurl = 'http://127.0.0.1:5000/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

# обращение к серверу с запросом из одного набора (его построили руками выше - data)
if __name__ == '__main__':
    response = send_json(data)
    print('предсказание',response)

предсказание 0.8408372740402843


Сделаем обработку массы запросов:

загрузим данные из news_test.csv

In [4]:
data = pd.read_csv('news_test.csv')

In [5]:
data.head(3)

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...


In [6]:
response = send_json(data[['title', 'text']].iloc[0,:])

In [7]:
response

0.014283113124919871

Сделаем N запросов и оценим время

In [9]:
N = 500

In [10]:
%%time
predictions = data[['title', 'text']].iloc[:N,:].apply(lambda x: send_json(x),1)

Wall time: 3.61 s


In [11]:
predictions.values[:5]

array([0.01428311, 0.95045439, 0.95262126, 0.0103803 , 0.94267861])

In [12]:
y_test = pd.read_csv('news_y_test.csv')

In [13]:
y_test = y_test.drop(('id'), axis=1)

In [14]:
y_test.head(3)

,label
0,0
1,1
2,0


In [15]:
from sklearn.metrics import roc_auc_score

In [16]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.7896774193548388